In [1]:
import PIL
import fitz
import pandas as pd 
from PIL import Image
import io
import re

In [2]:
doc = fitz.open('Data/Pathfinder_Bestiary_1.pdf') # abrindo o PDF


obs:
         
Pagina especifica com problemas:  pg 11(resolvido)  - 
    
problema - nome do monstro em 1 pagina e image na prox pagina. Colocar algo que caso encontre apenas o nome e não a imagem, pegar a imagem da proxima pagina.  - Resolvido
    
problema - pagina que não tem nada como por exemplo 9. colocar algo que caso as 2 funcoes não funcionem ignorar o resultado.  - Resolvido

problema - 2 monstros na mesma pagina pg 16  - Resolvido

problema - monstro com subcategoria como dragões, pg 92 e 93 -  Resolvido

# Tratamenteo para pegar imagem

In [3]:
def req_image (argumento):
    '''recebe um int(numero da pagina), substitue este valor e extrai a imagem da pg em questão'''
    page_image = doc[argumento] # salvando a pagina [numero da pagina] como page
    
    list_image = page_image.get_images() #  lista de imagens da pagina escolhida

    for item in list_image: 
            if item[3] >100 and item[3] != 1603 and item[2] >150 and item[8] == 'DCTDecode' :   # escolha da imagem pelo tamanho, quarto parametro acima de 100 e diferente de 1603, oitvo parametro da imagem igual a DCTDecode 
                img_creature = item
            else:
                ''
    imagem_isolada = doc.extract_image(img_creature[0])['image']
    
    return imagem_isolada
    

In [164]:
# Imagem_pronta = PIL.Image.open(io.BytesIO(imagem_isolada))  # para testar se a função está funcionando.
# Imagem_pronta

In [165]:

# page_image = doc[269] # salvando a pagina [numero da pagina] como page
# 
# list_image = page_image.get_images() #  lista de imagens da pagina escolhida
# 
# print(list_image)
# 
# for item in list_image: 
#         if item[3] >100 and item[3] != 1603 and item[2] >150 and item[8] == 'DCTDecode' :   # escolha da imagem pelo tamanho, quarto parametro acima de 100 e diferente de 1603, oitvo parametro da imagem igual a DCTDecode 
#             img_creature = item
#             print(item)
#         else:
#             ''
# imagem_isolada = doc.extract_image(img_creature[0])['image']
# 
# imagem_isolada
# 
# 

# Tratamento para pegar nome do monstro

In [4]:
def req_text(argument):
    
    '''Função recebe um int(numero da pagina) e retorna o nome limpo que está com a segunda coordenada de 135.28799438476562  '''
    
    page_text = doc[argument]  # salvando a pagina [numero da pagina] como page_text 

    words = page_text.get_text("words")  # extraindo todas as palavras da pagina 1
    
    lista_nomes = []
    
    for i in words:   # for para pegar o nomes do monstro na pagina. So funciona se o nome estiver com o segundo valor de 135.28799438476562
        
        for image in i:
            if image == 135.28799438476562:
                
                x = i[4]
                
                lista_nomes.append(x+' ')
            
            else:
                ''
    
    lista_nomes_sujos =''.join(lista_nomes)  
    
    
    if re.findall('.*?\d+/?\d?',lista_nomes_sujos) != []: # if para separar os nomes quando aparece algum numero
        lista_nomes_limpa = re.findall('.*?\d+/?\d?',lista_nomes_sujos)
    
    else: # else para selecionar os nomes que não tem numero
        lista_nomes_limpa = re.findall('\w*.\w*',lista_nomes_sujos)
    
    lista_nomes_limpa2 = [n.replace('/','_') for n in lista_nomes_limpa ]    
    
    return lista_nomes_limpa2



In [143]:
# x = req_text(92)  # para testar se a função está funcionando
# x
# name_monster = str(req_text(16)[0])
# name_monster

# Funcao para salvar a imagem com o nome certo

In [5]:

def img_names_b1 (pg):
    '''Funcao recebe o numero da pagina, roda a função para pegar o nome do monstro. 
    caso o nome do monstro não seja um grupo vzio, tenta rodar a função para pegar a image. Caso a função da imagem de erro, pega a imagem da proxima pagina. e salva na pasta escolhida.
    
    Caso o nome do monstro seja vazio, não tentar pegar mais nada e retorna a menssagem 'Vazio' '''   
    

    if req_text (pg) != []: # if para filtrar quando há nome na pagina
        names = (req_text (pg))    

        for item in names:   # rodando este For para pegar cada item separado quando ha mais de 1 monstro por pagina
            item = str(item)
            name_monster = item
            
        #name_monster = str(req_text(pg)[0])  # funcao req_text para pegar o nome do monstro e tranfomando o nome em str
            try:
                    imagem_pronta = PIL.Image.open(io.BytesIO(req_image(pg)))# abrindo a imagem gerada pela funcao req_image
                
                    imagem_pronta.save('C:/Users/Escanor/Desktop/Iron HAck/Projetos/Projeto obrigatorio/Pathfinder/img_name_b1/'+name_monster+'.jpg')
                
                    print ('imagem salva com o nome de :'+name_monster)
                                              
            except:
                    imagem_pronta = PIL.Image.open(io.BytesIO(req_image(pg+1)))# abrindo a imagem gerada pela funcao req_image
                
                    imagem_pronta.save('C:/Users/Escanor/Desktop/Iron HAck/Projetos/Projeto obrigatorio/Pathfinder/img_name_b1/'+name_monster+'.jpg')
                
                    return 'imagem da proxima pagina salva com o nome de :'+ name_monster

    
    elif req_text (pg) == []: # elif para filtrar quando não ha nome da pagina
        return 'vazio'
        
    
    else: # else para ajudar caso de algum erro em algum lugar estranho. 
        return 'erro'
    
    return 'image(s) Salva'


In [6]:
for i in range (7,289):
    img_names_b1(i)

imagem salva com o nome de :AASIMAR CR 1_2
imagem salva com o nome de :ABOLETh CR 7
imagem salva com o nome de :ASTRAL DEVA CR 14
imagem salva com o nome de :PLAnETAR CR 16
imagem salva com o nome de :AnIMATED OBjECT CR 3
imagem salva com o nome de :AnkhEg CR 3
imagem salva com o nome de :ARMY ANT SWARM CR 5
imagem salva com o nome de : GIANT ANT CR 2
imagem salva com o nome de :gORILLA CR 2
imagem salva com o nome de : DIRE APE (gIgAnTOPIThECUS) CR 3
imagem salva com o nome de :hOUnD ARChOn CR 4
imagem salva com o nome de :LAnTERn ARChOn CR 2
imagem salva com o nome de :TRUMPET ARChOn CR 14
imagem salva com o nome de :ASSASSIn VInE CR 3
imagem salva com o nome de :BRALAnI CR 6
imagem salva com o nome de :ghAELE CR 13
imagem salva com o nome de :LILLEnD CR 7
imagem salva com o nome de :BARGHEST CR 4
imagem salva com o nome de :BASIDIROnD CR 5
imagem salva com o nome de :BASILISk CR 5
imagem salva com o nome de :DIRE BAT CR 2
imagem salva com o nome de : BAT SWARM CR 2
imagem salva com 

In [142]:
#  page_text = doc[92]  # salvando a pagina [numero da pagina] como page_text 
#  words = page_text.get_text("words") 